In [1]:
import pandas as pd

Загрузка датасета, преимущественно содержащего аномалии (датасет 1)

In [2]:
with open('/home/lyumos/PycharmProjects/anomalies-detection-project/datasets processing/network characteristics dataset/abnormal traffic.csv', 'r') as file:
    df1 = pd.read_csv(file)
df1.head()

,protocol,flow_duration,total_forward_packets,total_backward_packets,total_forward_packets_length,total_backward_packets_length,forward_packet_length_mean,backward_packet_length_mean,forward_packets_per_second,backward_packets_per_second,forward_iat_mean,backward_iat_mean,flow_iat_mean,flow_packets_per_seconds,flow_bytes_per_seconds,label
0,17,2468,4,0,1580.0,0.0,395.0,0.0,1620.745543,0.0,822.666667,0.0,822.666667,1620.745543,6.401945e+05,DrDoS_DNS
1,17,133,4,0,5888.0,0.0,1472.0,0.0,30075.187970,0.0,44.333333,0.0,44.333333,30075.187970,4.427068e+07,DrDoS_DNS
2,17,33509,200,0,88000.0,0.0,440.0,0.0,5968.545764,0.0,168.386935,0.0,168.386935,5968.545764,2.626160e+06,DrDoS_DNS
3,17,288495,200,0,88000.0,0.0,440.0,0.0,693.252916,0.0,1449.723618,0.0,1449.723618,693.252916,3.050313e+05,DrDoS_DNS
4,17,9,2,0,2062.0,0.0,1031.0,0.0,222222.222222,0.0,9.000000,0.0,9.000000,222222.222222,2.291111e+08,DrDoS_DNS


Обработка датасета 1

In [3]:
df1.drop(df1.columns[0], axis=1, inplace=True)
df1.columns = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s', 'Fwd IAT Mean', 'Bwd IAT Mean', 'Flow IAT Mean', 'Flow Packets/s', 'Flow Bytes/s', 'Label']
df1.loc[(df1.Label == 'DrDoS_DNS'), 'Label'] = 1
df1.loc[(df1.Label == 'BENIGN'), 'Label'] = 0
df1.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Bwd Packet Length Mean,Fwd Packets/s,Bwd Packets/s,Fwd IAT Mean,Bwd IAT Mean,Flow IAT Mean,Flow Packets/s,Flow Bytes/s,Label
0,2468,4,0,1580.0,0.0,395.0,0.0,1620.745543,0.0,822.666667,0.0,822.666667,1620.745543,6.401945e+05,1
1,133,4,0,5888.0,0.0,1472.0,0.0,30075.187970,0.0,44.333333,0.0,44.333333,30075.187970,4.427068e+07,1
2,33509,200,0,88000.0,0.0,440.0,0.0,5968.545764,0.0,168.386935,0.0,168.386935,5968.545764,2.626160e+06,1
3,288495,200,0,88000.0,0.0,440.0,0.0,693.252916,0.0,1449.723618,0.0,1449.723618,693.252916,3.050313e+05,1
4,9,2,0,2062.0,0.0,1031.0,0.0,222222.222222,0.0,9.000000,0.0,9.000000,222222.222222,2.291111e+08,1


Загрузка датасета, содержащего нормальный трафик (датасет 2)

In [4]:
with open('/home/lyumos/PycharmProjects/anomalies-detection-project/datasets processing/network characteristics dataset/benign traffic.csv', 'r') as file:
    df2 = pd.read_csv(file)
df2.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Bwd Packet Length Mean,Fwd Packets/s,Bwd Packets/s,Fwd IAT Mean,Bwd IAT Mean,Flow IAT Mean,Flow Packets/s,Flow Bytes/s,Label
0,46870,1,1,44,240,44.0,240.0,21.335609,21.335609,0.0,0.0,46870.0,42.671218,6059.312993,BENIGN
1,62958,1,1,48,80,48.0,80.0,15.883605,15.883605,0.0,0.0,62958.0,31.767210,2033.101433,BENIGN
2,66230,1,1,50,162,50.0,162.0,15.098898,15.098898,0.0,0.0,66230.0,30.197796,3200.966329,BENIGN
3,398250,1,1,55,213,55.0,213.0,2.510986,2.510986,0.0,0.0,398250.0,5.021971,672.944131,BENIGN
4,23544,1,1,43,206,43.0,206.0,42.473666,42.473666,0.0,0.0,23544.0,84.947333,10575.942920,BENIGN


Обработка датасета 2

In [5]:
df2.columns = ['Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s', 'Fwd IAT Mean', 'Bwd IAT Mean', 'Flow IAT Mean', 'Flow Packets/s', 'Flow Bytes/s', 'Label']
df2.loc[(df2.Label == 'BENIGN'), 'Label'] = 0
df2.head()

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Mean,Bwd Packet Length Mean,Fwd Packets/s,Bwd Packets/s,Fwd IAT Mean,Bwd IAT Mean,Flow IAT Mean,Flow Packets/s,Flow Bytes/s,Label
0,46870,1,1,44,240,44.0,240.0,21.335609,21.335609,0.0,0.0,46870.0,42.671218,6059.312993,0
1,62958,1,1,48,80,48.0,80.0,15.883605,15.883605,0.0,0.0,62958.0,31.767210,2033.101433,0
2,66230,1,1,50,162,50.0,162.0,15.098898,15.098898,0.0,0.0,66230.0,30.197796,3200.966329,0
3,398250,1,1,55,213,55.0,213.0,2.510986,2.510986,0.0,0.0,398250.0,5.021971,672.944131,0
4,23544,1,1,43,206,43.0,206.0,42.473666,42.473666,0.0,0.0,23544.0,84.947333,10575.942920,0


Объединение датасетов

In [6]:
df = pd.concat([df1, df2])
df = df.sample(frac=1).reset_index(drop=True)
df.Label.value_counts()

Label
0    192067
1     32797
Name: count, dtype: int64

In [7]:
df.drop(df[df['Label'] == 0].index[:101438], inplace=True)
df.Label.value_counts()

Label
0    90629
1    32797
Name: count, dtype: int64